In [1]:
#Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv(r"E:\Study\ML\Digit_recognizer\train.csv")
test = pd.read_csv(r"E:\Study\ML\Digit_recognizer\test.csv")

In [3]:
#Divide training and val set in 10% ratio
X_val = train.iloc[:4200,1:]
Y_val = train.iloc[:4200,0]
X_train = train.iloc[4200:,1:]
Y_train = train.iloc[4200:,0]

X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_val = np.array(X_val)
Y_val = np.array(Y_val)

#Feature scaling
X_train = X_train/255
X_train = X_train.T

X_val = X_val/255
X_val = X_val.T

In [4]:
Y_train_new = Y_train
Y_val_new = Y_val

In [5]:
def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y
Y_train = one_hot(Y_train)
Y_val = one_hot(Y_val)
print(Y_train.shape)

(10, 37800)


In [6]:
print(X_train.shape)
print(X_val.shape)

(784, 37800)
(784, 4200)


In [7]:
#Sigmoid function
def softmax(Z):
    ans = np.exp(Z) / sum(np.exp(Z))
    cache = Z
    return ans,Z

#relu function
def relu(Z):
    rel = np.maximum(Z,0)
    cache = Z
    return rel,Z
    

In [63]:
#Initializing parameters
def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims) # number of layers in the network

    for l in range(1, L):
        parameters['W'+str(l)] = np.random.rand(layer_dims[l],layer_dims[l-1]) - 0.5
        parameters['b'+str(l)] = np.zeros((layer_dims[l],1))
        
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l - 1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [64]:
#Forward propagation
def linear_forward(A, W, b):
    Z = np.dot(W,A) + b
    cache = (A, W, b)
    
    return Z, cache

In [65]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "softmax":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = softmax(Z)
        
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(A_prev,W,b)
        A, activation_cache = relu(Z)
        
    cache = (linear_cache, activation_cache)

    return A, cache

In [66]:
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev,parameters["W"+str(l)],parameters["b"+str(l)],"relu")
        caches.append(cache)
        
    AL, cache = linear_activation_forward(A,parameters["W"+str(L)],parameters["b"+str(L)],"softmax")
    caches.append(cache)
          
    return AL, caches

In [67]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]

    cost = (-1/m)*np.sum(np.sum(Y*np.log(AL),axis=1),axis=0)
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).

    
    return cost

In [68]:
#Back propagation
def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m)*np.dot(dZ,A_prev.T)
    db = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    
    return dA_prev, dW, db

In [69]:
def softmax_backward(dA,activation_cache):
    A, cache =softmax(activation_cache)
    ans = dA*A*(1-A)
    return ans
    
def relu_backward(dA,activation_cache):
    ans = dA*(activation_cache>0)
    return ans
    

In [70]:
def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        
        
    elif activation == "softmax":
        dZ = softmax_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        
    
    return dA_prev, dW, db

In [71]:
def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "softmax" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    (Aprev, W, B), Z = caches[L-1]
    dZ = AL - Y
    grads["dA"+str(L-1)] = np.dot(W.T,dZ)
    grads["dW"+str(L)] = (1/m)*np.dot(dZ,Aprev.T)
    grads["db"+str(L)] = (1/m)*np.sum(dZ,axis=1,keepdims=True)
    
    # YOUR CODE ENDS HERE
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    #(approx. 5 lines)
    # current_cache = ...
    # dA_prev_temp, dW_temp, db_temp = ...
    # grads["dA" + str(L-1)] = ...
    # grads["dW" + str(L)] = ...
    # grads["db" + str(L)] = ...
    # YOUR CODE STARTS HERE
    #current_cache = caches[L-1]
    #grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "softmax")
    
    # YOUR CODE ENDS HERE
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        #(approx. 5 lines)
        # current_cache = ...
        # dA_prev_temp, dW_temp, db_temp = ...
        # grads["dA" + str(l)] = ...
        # grads["dW" + str(l + 1)] = ...
        # grads["db" + str(l + 1)] = ...
        # YOUR CODE STARTS HERE
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        
        # YOUR CODE ENDS HERE

    return grads

In [72]:
def update_parameters(params, grads, learning_rate):

    parameters = params.copy()
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate*grads["dW"+str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate*grads["db"+str(l+1)]
    return parameters

In [81]:
layers_dims = [784,256,84,32,10]

In [82]:
def L_layer_model(X, Y, layers_dims, learning_rate = 0.0075, num_iterations = 3000, print_cost=False):
    """
    Implements a L-layer neural network: [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID.
    
    Arguments:
    X -- data, numpy array of shape (num_px * num_px * 3, number of examples)
    Y -- true "label" vector (containing 0 if cat, 1 if non-cat), of shape (1, number of examples)
    layers_dims -- list containing the input size and each layer size, of length (number of layers + 1).
    learning_rate -- learning rate of the gradient descent update rule
    num_iterations -- number of iterations of the optimization loop
    print_cost -- if True, it prints the cost every 100 steps
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """

    np.random.seed(1)
    costs = []                         # keep track of cost
    
    parameters = initialize_parameters_deep(layers_dims)
    

    for i in range(0, num_iterations):

        AL, caches = L_model_forward(X,parameters)
        
        cost = compute_cost(AL,Y)
        
        grads = L_model_backward(AL,Y,caches)
        
        parameters = update_parameters(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0 or i == num_iterations - 1:
            print("Cost after iteration {}: {}".format(i, np.squeeze(cost)))
        if i % 100 == 0 or i == num_iterations:
            costs.append(cost)
    
    return parameters, costs

In [90]:
parameters, costs = L_layer_model(X_train, Y_train, layers_dims, num_iterations = 2000, print_cost = True, learning_rate=0.1)

Cost after iteration 0: 37.16095331299359
Cost after iteration 100: 1.6505334599645276
Cost after iteration 200: 1.0628561184925498
Cost after iteration 300: 0.6506839415399802
Cost after iteration 400: 0.4636543152548546
Cost after iteration 500: 0.3794192576753898
Cost after iteration 600: 0.3313291281333746
Cost after iteration 700: 0.29529123279237335
Cost after iteration 800: 0.2712522924481659
Cost after iteration 900: 0.25152917490265964
Cost after iteration 1000: 0.2353794350853048
Cost after iteration 1100: 0.22187359349626767
Cost after iteration 1200: 0.2094483770488963
Cost after iteration 1300: 0.1989259580180613
Cost after iteration 1400: 0.18902200947566541
Cost after iteration 1500: 0.1801748021944407
Cost after iteration 1600: 0.17215351020583677
Cost after iteration 1700: 0.16473869278266692
Cost after iteration 1800: 0.15784551425997898
Cost after iteration 1900: 0.15145394868802808
Cost after iteration 1999: 0.14553681427487306


In [91]:
def predictions(AL):
    return np.argmax(AL,0)

def make_predictions(parameters, X):
    AL, caches = L_model_forward(X, parameters)
    preds = predictions(AL)
    return preds

#Measuring accuracy
def accuracy(predictions,Y):
    return np.sum(predictions == Y) / Y.size

In [92]:
preds = make_predictions(parameters, X_train)

In [93]:
print(accuracy(preds,Y_train_new))

0.9561111111111111


In [94]:
val_preds = make_predictions(parameters, X_val)

In [95]:
print(accuracy(val_preds,Y_val_new))

0.9371428571428572
